# 🧠 Predict Exam Scores with FastAPI in Google Colab

In this notebook, you'll:
- 📊 **Part 1**: Generate and train a machine learning model
- 🌐 **Part 2**: Serve it with FastAPI in the background using `uvicorn`
- 🧪 **Part 3**: Test the live API from Colab using HTTP requests

<a href="https://colab.research.google.com/github/Stan-Pugsley/is_4487_base/blob/main/Reading-PracticeScripts/week14_practice_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 📊 Part 1: Generate and Train the Model
**Goal:** Create synthetic data, train a linear regression model, and save it for use in the API.

In [ ]:
# Install package needed for saving the model
!pip install joblib

In [ ]:
# Step 1: Generate synthetic data and train a model
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import joblib

# Generate data
np.random.seed(42)
hours = np.random.uniform(1, 10, 100)
noise = np.random.normal(0, 5, 100)
scores = 10 * hours + 30 + noise

# Create DataFrame
data = pd.DataFrame({'Hours_Studied': hours, 'Exam_Score': scores})
X = data[['Hours_Studied']]
y = data['Exam_Score']

# Train/test split and model training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
print(f"Model R² Score: {r2_score(y_test, y_pred):.2f}")

# Save model
joblib.dump(model, 'exam_score_model.pkl')

## 🌐 Part 2: Deploy the Model as an API
**Goal:** Use FastAPI to create a REST API and expose it publicly using ngrok.

**Step 1: Install required packages and set your ngrok authtoken**

1. Sign up at https://dashboard.ngrok.com/signup if you haven't already.
2. Copy your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
3. Paste it into the cell below where it says `YOUR_AUTHTOKEN`.

In [ ]:
!pip install fastapi uvicorn nest-asyncio pyngrok

In [ ]:
from pyngrok import ngrok
# Paste your ngrok auth token below
ngrok.set_auth_token("2vKth5KZ3aVArYtk01cC83dAAww_532dvLUksZ41Hun9pBe7W")  # ← Replace this with your token

In [ ]:
# Step 2: Create and launch the FastAPI app in the background
import nest_asyncio
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel
import joblib
import numpy as np
import threading

# Load the saved model
model = joblib.load("exam_score_model.pkl")

# Define the FastAPI app
app = FastAPI()
nest_asyncio.apply()

# Define input data model
class StudyInput(BaseModel):
    Hours_Studied: float

# Define prediction endpoint
@app.post("/predict")
def predict(input_data: StudyInput):
    prediction = model.predict(np.array([[input_data.Hours_Studied]]))[0]
    return {"Predicted_Exam_Score": prediction}

# Create public URL
public_url = ngrok.connect(8000)
print(f"🚀 FastAPI is running at: {public_url}/docs")

# Run FastAPI in background
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

thread = threading.Thread(target=run)
thread.start()

## 🧪 Part 3: Test the Live API
**Goal:** Send a POST request to the `/predict` endpoint and get the predicted exam score.

In [ ]:
# Step 3: Make a test request to your API
# Replace <your-ngrok-url> with the printed URL above
import requests

url = "ADD_URL_HERE_FROM_ABOVE/predict"  # Example: https://xxxx.ngrok.io/predict
response = requests.post(url, json={"Hours_Studied": 6})
print("Status Code:", response.status_code)
print("Response:", response.json())